In [91]:
import pandas as pd
import numpy as np

In [92]:
df = pd.read_excel("data.xlsx", sheet_name="Online Retail")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [93]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [94]:
customer_count = df.CustomerID.nunique()
customer_count

4372

In [95]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


In [96]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [97]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

df["Total Price"] = df["Quantity"] * df["UnitPrice"]

In [98]:
df.drop(columns=["Quantity", "UnitPrice"], inplace=True)

In [99]:
retail_by_customer = df.groupby("CustomerID")["Total Price"].sum().reset_index()
retail_by_customer.sort_values("Total Price", ascending=False)

,CustomerID,Total Price
1703,14646.0,279489.02
4233,18102.0,256438.49
3758,17450.0,187482.17
1895,14911.0,132572.62
55,12415.0,123725.45
...,...,...
125,12503.0,-1126.00
3870,17603.0,-1165.30
1384,14213.0,-1192.20
2236,15369.0,-1592.49


In [110]:
NO_purchases_by_customer = df.groupby("CustomerID")["InvoiceNo"].count().reset_index()
NO_purchases_by_customer.rename(columns={"InvoiceNo": "Purchase count"}, inplace=True)
NO_purchases_by_customer.sort_values("Purchase count", ascending=False)

,CustomerID,Purchase count
4042,17841.0,7983
1895,14911.0,5903
1300,14096.0,5128
330,12748.0,4642
1674,14606.0,2782
...,...,...
1318,14119.0,1
1046,13747.0,1
2400,15590.0,1
2379,15562.0,1


In [118]:
max_purchase_date = df["InvoiceDate"].max()

last_purchase_df = df.groupby("CustomerID")["InvoiceDate"].max().reset_index()

last_purchase_df["Days since last purchase"] = (
    max_purchase_date - last_purchase_df["InvoiceDate"]
).dt.days

customer_last_purchase_df = last_purchase_df[["CustomerID", "Days since last purchase"]]
customer_last_purchase_df.sort_values("Days since last purchase", ascending=True)

,CustomerID,Days since last purchase
3784,17491.0,0
3392,16933.0,0
3113,16558.0,0
524,13026.0,0
3691,17364.0,0
...,...,...
4212,18074.0,373
359,12791.0,373
4096,17908.0,373
3129,16583.0,373


In [114]:
customer_df = retail_by_customer.merge(NO_purchases_by_customer, on="CustomerID").merge(
    customer_last_purchase_df, on="CustomerID"
)
customer_df.sort_values("Purchase count", ascending=False)

,CustomerID,Total Price,Purchase count,Days Since Last Purchase
4042,17841.0,40340.78,7983,1
1895,14911.0,132572.62,5903,0
1300,14096.0,57120.91,5128,3
330,12748.0,29072.10,4642,0
1674,14606.0,11713.85,2782,0
...,...,...,...,...
1318,14119.0,-19.90,1,353
1046,13747.0,79.60,1,373
2400,15590.0,-30.19,1,288
2379,15562.0,134.55,1,351
